# Ensemble

In [88]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from data import train, validation
from data import X, y, categorical, numerical

In [109]:
pred_train = pd.read_csv('predictions/train.csv', index_col=0)
pred_validation = pd.read_csv('predictions/validation.csv', index_col=0)

In [110]:
pred_train.head()

,0,1,2,3,4,7,10,13,14
0,262.264045,208.946981,224.181583,281.291241,278.872627,269.716098,224.181583,208.887056,323.687086
1,5.717018,2.380884,-4.723311,4.933619,5.268990,4.467527,-4.723311,1.475746,6.989618
2,18.588675,20.537849,23.338251,16.516517,15.281036,16.040660,23.338251,20.961534,15.731637
3,8.092857,37.149286,28.099011,18.594146,22.367187,19.804791,28.099011,37.406971,19.384777
4,25.287449,32.700862,28.449217,26.751070,25.666721,25.566143,28.449217,32.706583,24.926269


In [111]:
from sklearn.linear_model import LinearRegression

meta = StandardizedGridSearchCV(
    estimator = LinearRegression(),
    param_grid = {
        'positive': [True, False],
    },
)

meta.fit(pred_train, train[y])
meta.results

Fitting 10 folds for each of 2 candidates, totalling 20 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,split5_test_neg_mean_squared_error,...,split2_train_r2,split3_train_r2,split4_train_r2,split5_train_r2,split6_train_r2,split7_train_r2,split8_train_r2,split9_train_r2,mean_train_r2,std_train_r2
positive,,,,,,,,,,,,,,,,,,,,,
False,0.003578,0.000516,0.002704,0.000467,-189.042278,-89.530175,-104.911070,-158.680030,-132.072354,-102.764959,...,0.989413,0.989665,0.989536,0.989490,0.988420,0.988883,0.988125,0.988767,0.989126,0.000548
True,0.003665,0.000417,0.002932,0.000501,-252.803830,-200.613806,-282.187015,-319.687999,-503.463533,-241.114420,...,0.969618,0.969249,0.971256,0.969502,0.967372,0.968124,0.977824,0.967262,0.969676,0.002942


In [112]:
meta.results[['mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']]

,mean_train_neg_mean_squared_error,mean_test_neg_mean_squared_error
positive,,
False,-110.944770,-159.529522
True,-311.226621,-573.110912


In [117]:
meta.save('ensemble.p')

# Model evaluation

The ensemble model was evaluated against the best individual model on the validation set. The ensemble model shows a significant improvement in prediction variance with a small improvement in prediction bias. 

In [115]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

y_true = validation[y]

print('Best indivdual model:')
y_pred = pred_validation['0']
print('- mse =', mean_squared_error(y_true=y_true, y_pred=y_pred))
print('- mae =', mean_absolute_error(y_true=y_true, y_pred=y_pred))
print('- std =', (y_true - y_pred).std())

print()
print('Ensemble model:')
y_pred = meta.predict(pred_validation)
print('- mse =', mean_squared_error(y_true=y_true, y_pred=y_pred))
print('- mae =', mean_absolute_error(y_true=y_true, y_pred=y_pred))
print('- std =', (y_true - y_pred).std())

Best indivdual model:
- mse = 1265.690846715096
- mae = 18.181575382899513
- std = 125.55516520789725

Ensemble model:
- mse = 1233.3377391708627
- mae = 17.152032323838696
- std = 35.136029959150626
